# Gesture Recognition - Mukul Kumar
As a data scientist at a home electronics company our aim is to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user. Each gesture corresponds to a specific command:

1. Thumbs up:  Increase the volume
2. Thumbs down: Decrease the volume
3. Left swipe: 'Jump' backwards 10 seconds
4. Right swipe: 'Jump' forward 10 seconds  
5. Stop: Pause the movie

We have to analyse each video which is sequence of 30 images or frame and build a model which will correctly recognise ach command.

In [23]:
import numpy as np
import os
import PIL
import cv2
import datetime
import os
import matplotlib.pyplot as plt

In [24]:
!pip install keras

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


We set the random seed so that the results don't vary drastically.

In [26]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [27]:
train_doc = np.random.permutation(open('/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/datasets/Project_data/val.csv').readlines())

# batch size
batch_size = 64 

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [28]:
# function to resize and normalize the images in one batch. This can be reused in full batch case and also in the remaining data points (left after full batches) case

def send_batch_data(source_path, folder_list, batch_size, img_idx, x, y, z, t, batch, batch_data, batch_labels):
    for folder in range(batch_size): # iterate over the batch_size
        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
            image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            
            # Resizing the images
            if image.shape[1] == 160:
                image = cv2.resize(image[:,20:140,:],(y,z)).astype(np.float32)
            else:
                image = cv2.resize(image,(y,z)).astype(np.float32)

            #normalizing the images
            batch_data[folder,idx,:,:,0] = cv2.normalize(image[:,:,0], None, alpha=100, beta=255, 
                                                                 norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            batch_data[folder,idx,:,:,1] = cv2.normalize(image[:,:,1], None, alpha=100, beta=255, 
                                                                 norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            batch_data[folder,idx,:,:,2] = cv2.normalize(image[:,:,2], None, alpha=100, beta=255, 
                                                                 norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)         
        batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
    return batch_data, batch_labels


In [29]:
# Generator

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    # List of image numbers to use for a particular video
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    
    x = len(img_idx) # number of images in the seq
    y = 84 # image_height
    z = 84 # image_width
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,x,y,z,3)) 
            
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            # Calling the function which forms the batch data
            batch_data, batch_labels = send_batch_data(source_path, folder_list, batch_size, img_idx, x, y, z, t, 
                                                       batch, batch_data, batch_labels)
            
            yield batch_data, batch_labels # yielding the batch_data and the batch_labels

        
        # Code for the remaining data points which are left after full batches
        rem_img_size = len(folder_list) - batch_size * num_batches # Calculating the number of remaining images
        batch_num = num_batches # Updating batch number as the last batch, this is to index the remaining images
        # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_data = np.zeros((rem_img_size,x,y,z,3)) 
        batch_labels = np.zeros((rem_img_size,5)) # batch_labels is the one hot representation of the output
        
        # Calling the function which forms the batch data
        batch_data, batch_labels = send_batch_data(source_path, folder_list, rem_img_size, img_idx, x, y, z, t, 
                                                   batch_num, batch_data, batch_labels)
        yield batch_data, batch_labels # yielding the batch_data and the batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
# define train and validation path and number of epochs

curr_dt_time = datetime.datetime.now()
train_path = '/datasets/Project_data//train'
val_path = '/datasets/Project_data//val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 80 # choose the number of epochs
print ('# epochs =', num_epochs)

In [ ]:
# plot a sample video sequence (30 images)
import glob
from skimage import io

img_path = os.path.join(train_path, train_doc[2].strip().split(';')[0], '*')
img_path = glob.glob(img_path)

# access some element (a file) from the list
for i in range(0,29):
    image = io.imread(img_path[i])
    plt.imshow(image)
    plt.show()

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

## 1st Model - the Conv3d model.

In [ ]:
#set the image height, weight and numer of image

img_height = 84 
img_width = 84
num_img = 18

In [ ]:
# import libararies to build Conv3D model

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#Initialise the sequential model
model = Sequential()

# add layers to the model
model.add(Conv3D(32, kernel_size=(3, 3, 3),activation='relu',padding='same', input_shape=(num_img,img_height,img_width,3)))
model.add(BatchNormalization()),
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3),activation='relu', padding='same'))
model.add(Conv3D(64, kernel_size=(3, 3, 3),activation='relu', padding='same'))
model.add(BatchNormalization()),
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))

model.add(Conv3D(128, kernel_size=(3, 3, 3),activation='relu', padding='same'))
model.add(BatchNormalization()),
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))

model.add(Conv3D(256, kernel_size=(3, 3, 3),activation='relu', padding='same'))
model.add(BatchNormalization()),
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
# define optimizer
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)

#compile the model
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

#Print model summary
print (model.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
#call generator for training data
train_generator = generator(train_path, train_doc, batch_size)

#call generator for validation data
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
#Define model name
model_name = 'model_Conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

# directory to save model
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
#filepath to save models
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0,min_lr=0.0001)

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
#steps per epoch for train and validation sequence

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
#run the model
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

### Visualizing the results for Conv3D Model

In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(num_epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 2nd Model - the conv plus RNN model. Will use ResNet50 as transer learning

In [ ]:
from keras.applications import ResNet50

# Building the base_model with ResNet101
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(img_height,img_width,3))
x = base_model.output
x = Flatten()(x)
conv_model = Model(inputs=base_model.input, outputs=x)
    
for layer in base_model.layers:
    layer.trainable = False

RNN_model = Sequential()
RNN_model.add(TimeDistributed(conv_model,input_shape=(num_img,img_height,img_width,3)))
RNN_model.add(GRU(64, return_sequences=False))
RNN_model.add(Dense(64))
RNN_model.add(Dense(5,activation='softmax'))

# Optimizer and compilation of the model
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
RNN_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (RNN_model.summary())

In [ ]:
#Define model name
model_name = 'model_RNN' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

# directory to save model
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
#filepath to save models
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0,min_lr=0.0001)

callbacks_list = [checkpoint, LR]

In [ ]:
history = RNN_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

### Visualizing the results for RNN Model

In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(num_epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Conclusion
- The RNN model the train and validation accuaracy is 100% and 82%. Clearly the model is overfitting
- The Conv3D model with train accuracy of 91% and validation accuracy of 86% is better that RNN model.
- The final model is saved as below: